In [1]:
import json
from glob import glob
from multiprocessing import Pool
from pathlib import Path
import os
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from stexls.util import file_watcher
from pickle import dumps, loads, dump, load

from stexls.util.latex import parser

from stexls import *

In [2]:
%cd ~/Downloads

/home/marian/Downloads


In [3]:
parsed = parser.LatexParser('smglom/primes/source/balancedprime.en.tex')
parsed.parse()

In [4]:
for t in parsed.root.children:
    print(t)
    for c in t.children:
        print(c, *c.tokens)

[Environment name=[Node "mhmodnl"]]
[Environment name=[Node "definition"]] [Token begin=70 end=75 "A"] [Token begin=82 end=90 "balanced"] [Token begin=92 end=97 "prime"] [Token begin=98 end=104 "is a"] [Token begin=125 end=130 "prime"] [Token begin=132 end=138 "number"] [Token begin=139 end=220 "that is equal to the
  arithmetic mean of the nearest primes above and below."] [Token begin=220 end=339 "\[\eq{\nPrimeNumber{n},
      \ratdivide[frac]{\natplus{\nPrimeNumber{\natplus{n,1}},\nPrimeNumber{\natplus{n,1}}}}2}\]"] [Token begin=339 end=348 "where"] [Token begin=348 end=366 "$\nPrimeNumber{n}$"] [Token begin=366 end=374 "is the"] [Token begin=374 end=377 "$n$"] [Token begin=377 end=394 "th prime number."]


In [5]:
def parse_files(root=Path.home()/'Downloads/smglom', limit: int = -1):
    ww = file_watcher.WorkspaceWatcher(root, filter='*.tex')
    changes = ww.update()
    with Pool() as pool:
        parsed_files = list(tqdm(pool.imap_unordered(parse, list(changes.created | changes.modified)[:limit])))
    return parsed_files

In [6]:
use_cache = False
cachefile = Path('/tmp/cache.bin')
if cachefile.is_file() and use_cache:
    with open(cachefile, 'rb') as fd:
        parsed_files = load(fd)
else:
    parsed_files = parse_files()
    with open(cachefile, 'w+b') as fd:
        dump(parsed_files, fd)

Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/marian/.pyenv/versions/TF1/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/marian/.pyenv/versions/TF1/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/marian/.pyenv/versions/TF1/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marian/.pyenv/versions/TF1/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marian/.pyenv/versions/TF1/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-3:
  File "/home/marian/.pyenv/versions/TF1/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
   

KeyboardInterrupt: 

In [ ]:
for pf in parsed_files:
    if pf.errors:
        print(pf.errors)
        print()
        print('-'*80)
        print()

In [ ]:
with Pool() as pool:
    compiled_files = pool.map(StexObject.compile, tqdm(parsed_files))

In [ ]:
ok = list(filter(None, compiled_files))

In [ ]:
links, resolved_references = index_and_link(ok)

In [ ]:
for id, refs in resolved_references.items():
    for ref in refs:
        if isinstance(ref, DefSymbol):
            if ref.noverb or ref.noverbs:
                print(ref, ref.noverbs)

In [ ]:
resolved_references